<a href="https://colab.research.google.com/github/tomassalcedas/dataeng/blob/main/spark/examples/09-windows-function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Windows Function
- Window functions operate on a group of rows, referred to as a window, and calculate a return value for each row based on the group of rows.
- Window functions are useful for processing tasks such as calculating a moving average, computing a cumulative statistic, or accessing the value of rows given the relative position of the current row.

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').config('spark.ui.port', '4050').getOrCreate()

# Windows Function

In [4]:
# using inline table to prepare the data

qry = """CREATE OR REPLACE TEMPORARY VIEW employees AS SELECT * FROM VALUES("Lisa", "Sales", 10000, 35),
("Evan", "Sales", 32000, 38),
("Fred", "Engineering", 21000, 28),
("Alex", "Sales", 30000, 33),
("Tom", "Engineering", 23000, 33),
("Jane", "Marketing", 29000, 28),
("Jeff", "Marketing", 35000, 38),
("Paul", "Engineering", 29000, 23),
("Chloe", "Engineering", 23000, 25)
AS employees(name, dept, salary, age)"""

spark.sql(qry)
spark.table("employees").show()

+-----+-----------+------+---+
| name|       dept|salary|age|
+-----+-----------+------+---+
| Lisa|      Sales| 10000| 35|
| Evan|      Sales| 32000| 38|
| Fred|Engineering| 21000| 28|
| Alex|      Sales| 30000| 33|
|  Tom|Engineering| 23000| 33|
| Jane|  Marketing| 29000| 28|
| Jeff|  Marketing| 35000| 38|
| Paul|Engineering| 29000| 23|
|Chloe|Engineering| 23000| 25|
+-----+-----------+------+---+



In [5]:
# calculate dense_rank on salary
qry1 = """SELECT name, dept, salary, DENSE_RANK() OVER (PARTITION BY dept ORDER BY salary ROWS BETWEEN
    UNBOUNDED PRECEDING AND CURRENT ROW) AS dense_rank FROM employees;"""

spark.sql(qry1).show()

+-----+-----------+------+----------+
| name|       dept|salary|dense_rank|
+-----+-----------+------+----------+
| Fred|Engineering| 21000|         1|
|  Tom|Engineering| 23000|         2|
|Chloe|Engineering| 23000|         2|
| Paul|Engineering| 29000|         3|
| Jane|  Marketing| 29000|         1|
| Jeff|  Marketing| 35000|         2|
| Lisa|      Sales| 10000|         1|
| Alex|      Sales| 30000|         2|
| Evan|      Sales| 32000|         3|
+-----+-----------+------+----------+



In [6]:
# calculate min salary by dept
qry2 = """SELECT name, dept, salary, MIN(salary) OVER (PARTITION BY dept ORDER BY salary) AS min
    FROM employees;"""

spark.sql(qry2).show()

+-----+-----------+------+-----+
| name|       dept|salary|  min|
+-----+-----------+------+-----+
| Fred|Engineering| 21000|21000|
|  Tom|Engineering| 23000|21000|
|Chloe|Engineering| 23000|21000|
| Paul|Engineering| 29000|21000|
| Jane|  Marketing| 29000|29000|
| Jeff|  Marketing| 35000|29000|
| Lisa|      Sales| 10000|10000|
| Alex|      Sales| 30000|10000|
| Evan|      Sales| 32000|10000|
+-----+-----------+------+-----+



In [7]:
# same logic but using dataframes
from pyspark.sql.functions import min
from pyspark.sql.window import Window

df = spark.table("employees")
windowSpec = Window.partitionBy("dept").orderBy("salary")
df.withColumn("min", min("salary").over(windowSpec)).show()

+-----+-----------+------+---+-----+
| name|       dept|salary|age|  min|
+-----+-----------+------+---+-----+
| Fred|Engineering| 21000| 28|21000|
|  Tom|Engineering| 23000| 33|21000|
|Chloe|Engineering| 23000| 25|21000|
| Paul|Engineering| 29000| 23|21000|
| Jane|  Marketing| 29000| 28|29000|
| Jeff|  Marketing| 35000| 38|29000|
| Lisa|      Sales| 10000| 35|10000|
| Alex|      Sales| 30000| 33|10000|
| Evan|      Sales| 32000| 38|10000|
+-----+-----------+------+---+-----+



# Question

In [ ]:
# Q1
# Use windows functions to identify the higher salary by dept
# Create new column "highest_salary_dept" and assign TRUE/FALSE accordingly
# Identify the high salary of the company (including all the dept)
# Create new column "highest_salary_company" and assign TRUE/FALSE accordingly